In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from sklearn.metrics import (
    make_scorer,
    mean_absolute_error,
    mean_squared_error,
    r2_score
)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import  MLPRegressor
from sklearn.svm import SVR
import numpy as np

In [3]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

METRICS_REGRESSION = {
    "MAE": make_scorer(mean_absolute_error),
    "MSE": make_scorer(mean_squared_error),
    "RMSE": make_scorer(root_mean_squared_error),
    "R2": make_scorer(r2_score)
}

In [4]:
df = pd.read_csv("../cenario4_engine.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3207 entries, 0 to 3206
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   model_year  3207 non-null   int64  
 1   KM          3207 non-null   int64  
 2   HP          2578 non-null   float64
 3   Litros      2891 non-null   float64
 4   Cilindros   2705 non-null   float64
 5   price_eur   3207 non-null   int64  
dtypes: float64(3), int64(3)
memory usage: 150.5 KB


In [6]:
medias_colunas = df[['HP', 'Litros', 'Cilindros']].mean()
colunas = ['HP', 'Litros', 'Cilindros']

for coluna in colunas:
    df[coluna] = df[coluna].fillna(medias_colunas[coluna])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3207 entries, 0 to 3206
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   model_year  3207 non-null   int64  
 1   KM          3207 non-null   int64  
 2   HP          3207 non-null   float64
 3   Litros      3207 non-null   float64
 4   Cilindros   3207 non-null   float64
 5   price_eur   3207 non-null   int64  
dtypes: float64(3), int64(3)
memory usage: 150.5 KB


In [8]:
categorical_columns = df.select_dtypes(include='object').columns

for column in categorical_columns:
    df[column] = df[column].astype('category').cat.codes

In [9]:
X, y = df.drop("price_eur", axis=1), df["price_eur"]

In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

Random Forest

In [11]:
rf = RandomForestRegressor(random_state=1234)
scores_rf = cross_validate(rf, X, y, cv=splitter, scoring=METRICS_REGRESSION)
rf_scores = pd.DataFrame(scores_rf)
pd.DataFrame(rf_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_R2
0,1.716575,0.010579,13298.406919,4.253211e+09,52688.815183,0.365485


Redes Neuronais

In [12]:
nn = MLPRegressor(hidden_layer_sizes=(50,50), max_iter=200, random_state=1234)
scores_nn = cross_validate(nn, X, y, cv=splitter, scoring=METRICS_REGRESSION)
nn_scores = pd.DataFrame(scores_nn)
pd.DataFrame(nn_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_R2
0,1.942125,0.002367,19585.378315,4.962512e+09,58644.689186,0.27535


Support Vector Machine

In [13]:
svm = SVR()
scores_svm = cross_validate(svm, X, y, cv=splitter, scoring=METRICS_REGRESSION)
svm_scores = pd.DataFrame(scores_svm)
pd.DataFrame(svm_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_R2
0,0.255503,0.057557,24561.459617,5.945690e+09,67166.271036,-0.036542
